In [ ]:
# pandas numpy sklearn joblib livelossplot seaborn tqdm matplotlib keras_tqdm kiteconnect ipywidgets ta 
import threading
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
import joblib
import livelossplot
import seaborn as sns
from tqdm import tqdm
from matplotlib.pyplot import figure
from numpy import loadtxt
from stkdef60 import One_trian_stk_class
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import numpy as np
import pandas as pd
import xgboost
import math
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import skew
from collections import OrderedDict
from stkdef60 import One_trian_stk_class
import logging
import pickle
import matplotlib.pyplot as plt
from kiteconnect import KiteConnect
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ta import add_all_ta_features
import ta
logging.basicConfig(level=logging.ERROR)
# from pactools.grid_search import GridSearchCVProgressBar
homediraddr = '/home/hemangjoshi37a_gmail_com'
kite = KiteConnect(api_key="w19o0chuo929jxkp")
kite.login_url()
# plt.style.use("seaborn")
kite = joblib.load('kitefile.p')

In [ ]:
parameters_for_testing = {
   'estimator__colsample_bytree':[0.4,0.8],
   'estimator__gamma':[0.03,0.3],
   'estimator__min_child_weight':[1.5,10],
   'estimator__learning_rate':[0.1,0.07],
   'estimator__max_depth':[3,15],
   'estimator__n_estimators':[10000],
   'estimator__reg_alpha':[1e-5,  0.75],
   'estimator__reg_lambda':[1e-5, 0.45],
   'estimator__subsample':[0.6,0.95],
}

xgbmodel = xgboost.XGBRegressor(colsample_bytree=0.4,
                                gamma=0.5,
                                learning_rate=0.05,
                                max_depth=20,
                                min_child_weight=1.1,
                                n_estimators=10000,
                                reg_alpha=0.50,
                                gpu_id=0,
                                tree_method='gpu_hist',
                                reg_lambda=0.40,
                                subsample=0.6,
                                seed=50,
                                n_jobs=-1,
                                objective='reg:squarederror')

xx={'name':'TATASTEEL',
    'insttkn': 895745,
    'nbatch': 1,
    'nepochs': 5,
    'trainlen': 'all',
    'kite': kite}
trainit = One_trian_stk_class(xx['name'], xx['insttkn'], xx['nbatch'], xx['nepochs'], xx['trainlen'], xx['kite'])
trainit.load()
trainit.ta_xtrain_def()

reshapedxt = trainit.ta_x_train.reshape([trainit.ta_x_train.shape[0],trainit.ta_x_train.shape[1]*trainit.ta_x_train.shape[2]])
xtreshaped = trainit.ta_x_test.reshape([trainit.ta_x_test.shape[0],trainit.ta_x_train.shape[1]*trainit.ta_x_train.shape[2]])

multioutputregressor = MultiOutputRegressor(xgbmodel)

eval_set = [(reshapedxt, trainit.ta_y_train), (xtreshaped, trainit.ta_y_test)]

gsearch1 = GridSearchCV(estimator=multioutputregressor,
                        param_grid=parameters_for_testing,
                        n_jobs=8,
                        iid=False,
                        verbose=1,
                        scoring='neg_mean_squared_error')

gsearch1.fit(reshapedxt, trainit.ta_y_train,
            eval_metric=["error", "logloss"],
            eval_set=eval_set,
            verbose=False)
print(gsearch1.grid_scores_)
print('best params')
print(gsearch1.best_params_)
print('best score')
print(gsearch1.best_score_)
joblib.dump(gsearch1, "gsearch1.p") 

In [ ]:
joblib.dump(xtrain_df, "xtrain_fut.p")
joblib.dump(trainit.ta_y_train, "ytrain_fut.p")
joblib.dump(xtest_df, "xtest_fut.p")
joblib.dump(trainit.ta_y_test, "ytest_fut.p")

In [ ]:
trainit.dxtrain.save_binary("dxtrain.p")
trainit.dytrain.save_binary("dytrain.p")
trainit.dxtest.save_binary("dxtest.p")
trainit.dytest.save_binary("dytest.p")

In [ ]:
xtrain=joblib.load('xtrain_fut.p')
ytrain=joblib.load('ytrain_fut.p')
xtest=joblib.load('xtest_fut.p')
ytest=joblib.load('ytest_fut.p')
eval_set = [(xtrain, ytrain), (xtest,ytest)]

In [ ]:
xtrain_df = pd.DataFrame(xtrain).drop(columns=[4619]).to_numpy()
xtest_df = pd.DataFrame(xtest).drop(columns=[4619]).to_numpy()

In [ ]:
xtrain[100]

In [ ]:
ytrain[5000]

In [ ]:
xtest[940]

In [ ]:
ytest[940]

In [ ]:
ytrain_o=ytrain
ytest_o=ytest

In [ ]:
xx={'name':'USDINR20OCTFUT',
    'insttkn':690691,
    'nbatch':1,
    'nepochs':5,
    'trainlen':'all',
    'kite':kite}
trainit= One_trian_stk_class(xx['name'], xx['insttkn'],xx['nbatch'],xx['nepochs'],
                             xx['trainlen'],xx['kite'])
trainit.load()
trainit.ta_xtrain_def()

xtrain=trainit.ta_x_train.reshape([trainit.ta_x_train.shape[0],
                                   trainit.ta_x_train.shape[1]*trainit.ta_x_train.shape[2]])

xtest=trainit.ta_x_test.reshape([trainit.ta_x_test.shape[0],
                                  trainit.ta_x_train.shape[1]*trainit.ta_x_train.shape[2]])

ytrain = trainit.ta_y_train

eval_set = [(xtrain, trainit.ta_y_train), (xtest, trainit.ta_y_test)]

In [ ]:
percent=0.60

xtrain=xtrain[-math.ceil((xtrain.shape[0])*percent):]
ytrain=ytrain[-math.ceil((ytrain.shape[0])*percent):]
eval_set = [(xtrain, ytrain), (xtest,ytest)]
print('xtrain = '+str(xtrain.shape))            
print('ytrain = '+str(ytrain.shape))

print('xtest = '+str(xtest.shape))      
print('ytest = '+str(ytest.shape))

In [ ]:
ytrain = ytrain[:,:1]
ytest = ytest[:,:1]
print(ytrain.shape)
print(ytest.shape)

In [ ]:
nr=xgboost.XGBRegressor(
                 colsample_bytree=0.6,
                 gamma=0.5,
                 learning_rate=0.07,
                 max_depth=25,
                 min_child_weight=1.2,
                 n_estimators=1000,
                 gpu_id=0,
                 tree_method='gpu_hist',
                 objective='reg:squarederror',
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=70,
                 verbosity=1,
                 predictor = 'gpu_predictor',
                 #n_jobs=-1,
)

eval_set=[(xtrain,ytrain),(xtest,ytest)]

nr.fit(xtrain,ytrain,
        eval_metric=["error", "logloss"],
            eval_set=eval_set,
            verbose=True
      )
joblib.dump(nr, "xgbregressor18.p")

In [ ]:
multioutputregressor = MultiOutputRegressor(xgboost.XGBRegressor(
                colsample_bytree=0.4,
                 gamma=0.5,
                 learning_rate=0.1,
                 max_depth=10,
                 min_child_weight=1.3,
                 n_estimators=100,
                 gpu_id=0,
                 tree_method='gpu_hist',
                 objective='reg:squarederror',
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=15,
                 verbosity=1,
                 n_jobs=-1))

# multioutputregressor = MultiOutputRegressor(xgboost.XGBRegressor(objective='reg:squaredlogerror',
#                                                                  verbosity=1,
#                                                                  gpu_id=0,
#                                                                  n_jobs=-1
#                                                                  ),
#                                                                  n_jobs=-1
#                                                                  )

print(multioutputregressor.get_params())

In [ ]:
res1=multioutputregressor.fit(xtrain,ytrain,
                          eval_metric=["error","logloss"], 
                          eval_set=eval_set,
                          verbose=True)

joblib.dump(multioutputregressor, "multioutputregressor11.p")

In [ ]:
multioutputregressor.score(xtest, ytest)

In [ ]:
corrMatrix = trainit.talibdf.corr()
figure(num=None, figsize=(8, 8), dpi=150)
sns.heatmap(corrMatrix)
plt.show()

In [ ]:
pred0=nr.predict(xtest[0].reshape(1,-1))
act0=ytest[0][0]
diff=np.array(pred0[0])-np.array(act0)
print(diff)

In [ ]:
pred1=nr.predict(xtest[1].reshape(1,-1))
act1=ytest[1]
diff1=np.array(pred1[0])-np.array(act1)
print(diff1)

In [ ]:
pred2=nr.predict(xtest[2].reshape(1,-1))
act2=ytest[2]
diff2=np.array(pred2[0])-np.array(act2)
print(diff2)

In [ ]:
pred0=multioutputregressor.predict(trainit.ta_x_test[0].reshape(1,60*trainit.ta_x_test.shape[2]))
act0=trainit.ta_y_test[0]
diff=np.array(pred0[0])-np.array(act0)
difflist=diff.tolist()
table0=pd.DataFrame({'true':act0,'pred':pred0[0]})
table0.plot(figsize=(13, 6))

In [ ]:
pred1=multioutputregressor.predict(trainit.ta_x_test[21].reshape(1,60*trainit.ta_x_test.shape[2]))
act1=trainit.ta_y_test[21]
diff=np.array(pred1[0])-np.array(act1)
difflist1=diff.tolist()
table1=pd.DataFrame({'true':act1,'pred':pred1[0]})
table1.plot(figsize=(13, 6))

In [ ]:
pred2=multioutputregressor.predict(trainit.ta_x_test[25].reshape(1,60*trainit.ta_x_test.shape[2]))
act2=trainit.ta_y_test[25]
diff=np.array(pred2[0])-np.array(act2)
difflist2=diff.tolist()
table2=pd.DataFrame({'true':act2,'pred':pred2[0]})
table2.plot(figsize=(13, 6))

In [ ]:
# newdict={'table1_pred':table0['diff'].values.tolist(),'table2_pred':table1['diff'].values.tolist()} 
# hjdf=pd.DataFrame(newdict)
# hjdf.plot(figsize=(13, 6))

In [ ]:
sample_no=0
table2=pd.DataFrame({'true':trainit.ta_y_test[sample_no],'pred':multioutputregressor.predict(trainit.ta_x_test[sample_no].reshape(1,60*trainit.ta_x_test.shape[2]))[0]})
table2.plot(figsize=(13, 6))

In [ ]:
@interact
def shwo_gra(x=189):
    sample_no=x
#     sample_no=int(y)
    table2=pd.DataFrame({'true':trainit.ta_y_test[sample_no],'pred':multioutputregressor.predict(trainit.ta_x_test[sample_no].reshape(1,60*trainit.ta_x_test.shape[2]))[0]})
    print(table2.head())
    actar=np.array(trainit.ta_y_test[sample_no])
    predar=np.array(multioutputregressor.predict(trainit.ta_x_test[sample_no].reshape(1,60*trainit.ta_x_test.shape[2]))[0])
    diffdf=pd.DataFrame(actar-predar)
    diffdf.plot()
    diffa=(np.mean(np.abs(actar-predar)))
    print('acc : '+ str(diffa))
    return table2.plot(figsize=(10, 6))
#     return table2
#     return trainit.talibdf.loc[trainit.talibdf[column] = x]

In [ ]:
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna

df =trainit.df

df = add_all_ta_features(
    df, open="open", high="high", low="low", close="close", volume="volume")